In [1]:
import psycopg2
import pandas as pd
import sqlalchemy

In [11]:
conn = psycopg2.connect(host="127.0.0.1", port="5432", database="Adventureworks", 
                            user="postgres", password="postgres")
cur = conn.cursor()

In [3]:
query = f'''
        select * from pg_trigger;
'''
df = pd.read_sql_query(query, conn)
df.head()

/Users/denis/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,oid,tgrelid,tgparentid,tgname,tgfoid,tgtype,tgenabled,tgisinternal,tgconstrrelid,tgconstrindid,tgconstraint,tgdeferrable,tginitdeferred,tgnargs,tgattr,tgargs,tgqual,tgoldtable,tgnewtable
0,17774,16450,0,RI_ConstraintTrigger_a_17774,1654,9,O,True,16460,17711,17773,False,False,0,,[],None,None,None
1,17775,16450,0,RI_ConstraintTrigger_a_17775,1655,17,O,True,16460,17711,17773,False,False,0,,[],None,None,None
2,17776,16460,0,RI_ConstraintTrigger_c_17776,1644,5,O,True,16450,17711,17773,False,False,0,,[],None,None,None
3,17777,16460,0,RI_ConstraintTrigger_c_17777,1645,17,O,True,16450,17711,17773,False,False,0,,[],None,None,None
4,17779,16648,0,RI_ConstraintTrigger_a_17779,1654,9,O,True,16591,17471,17778,False,False,0,,[],None,None,None


In [5]:
cur.execute('''
           create table if not exists  customer 
            (customer_id int,
             store_id int,
             first_name varchar,
             last_name varchar)
            ''')


cur.execute('''
           insert into   customer
            values(1,1,'Mike','Mike');
            ''')


cur.execute('''
           select* from customer
        
            ''')

results = cur.fetchall()
pd.DataFrame(results, columns=['customer_id', 'store_id', 'first_name', 'last_name'])

,customer_id,store_id,first_name,last_name
0,1,1,Mike,Mike
1,1,1,Mike,Mike


### создание табилцы логов

In [4]:
cur.execute('''
            create table logs (id serial primary key, 
            notes text, 
            added timestamp);
            ''')

cur.execute('''
            create or replace function add_to_log() returns trigger as $$
            declare
                str_total text;
            begin
                if tg_op = 'INSERT' or tg_op = 'UPDATE' then
                str_total := tg_relname || ' ' || new || ' ' || tg_op;
                insert into logs(notes, added) values (str_total, now());
                return new;
                elseif tg_op = 'DELETE' then
                str_total := tg_relname || ' ' || old || ' ' || tg_op;
                insert into logs(notes,added) values (str_total, now());
                return old;
                end if;
            end;
            $$ language plpgsql;

            ''')

In [6]:
cur.execute('''
            create trigger tr_customer
            after insert or update or delete on public.customer 
            for each row execute function add_to_log();

            ''')

In [8]:
cur.execute('''
            insert into customer values (999,10,'fntest','lntest');
            ''')


cur.execute('''
            delete from customer where customer_id  = 999;
            ''')

cur.execute('''
           select * from logs;
            ''')

results = cur.fetchall()
pd.DataFrame(results, columns=['id', 'notes', 'added'])

,id,notes,added
0,1,"customer (999,10,fntest,lntest) INSERT",2023-07-22 19:14:19.566122
1,2,"customer (999,10,fntest,lntest) DELETE",2023-07-22 19:14:19.566122
2,3,"customer (999,10,fntest,lntest) INSERT",2023-07-22 19:14:19.566122
3,4,"customer (999,10,fntest,lntest) DELETE",2023-07-22 19:14:19.566122


### если вы также хотите увидеть, к какой таблице применяется каждый триггер 

In [12]:
query = f'''
        select tgrelid::regclass, tgname from pg_trigger;
'''
df = pd.read_sql_query(query, conn)
df.head()

,tgrelid,tgname
0,person.stateprovince,RI_ConstraintTrigger_a_17774
1,person.stateprovince,RI_ConstraintTrigger_a_17775
2,person.address,RI_ConstraintTrigger_c_17776
3,person.address,RI_ConstraintTrigger_c_17777
4,production.product,RI_ConstraintTrigger_a_17779


In [9]:
conn.rollback()
conn.close()